# 3. Работа с Big Data

## 3.1. Spark SQL (каталог, метаданные)

Каталог (Catalog) — это абстракция в Spark SQL, которая предоставляет API для:
- Управления базами данных (databases)
- Работы с таблицами (tables)
- Доступа к представлениям (views)
- Хранения метаданных (схем, типов данных, партиций)

Встроенный каталог vs Hive Metastore
| Тип | Временная таблица | Постоянная таблица |
|- |- |- |
| Видимость | Только в текущей сессии Spark | Доступна всем сессиям |
| Хранение метаданных | В памяти | В Hive Metastore |
| Хранение данных | Зависит от источника | Сохраняется на HDFS/S3 |

## 3.2. Structured Streaming (основы)

## 3.2. Structured Streaming (основы)

## 3.3. Оптимизация запросов (Explain, Tungsten)

## 3.4. Работа с GraphFrames

## 3.5. Интеграция с Python (Pandas, NumPy)